# CSV data wrangling using numpy and pandas

### Purpose: 
The purpose of this notebook is to extract the informatin for year 2009 from csv file which contain Medicare Outpatient claims data extracted from synpuf data source (https://www.cms.gov/Research-Statistics-Data-and-Systems/Downloadable-Public-Use-Files/SynPUFs/DE_Syn_PUF) using pandas or numpy libaries.

### Input: 
A csv file which contains outpatient medicare claims data.

### Output:
A csv file which has extracted information from input file for year 2009 including Patient ID, Total claims in the year, Total Costs of outpatient care, Number of physician visited, Most frequent claim diagnosis, Most frequent admitting diagnosis

### Steps:
  Fist import all the relavant libaries,read input csv file,check few rows of csv file, and data type for each variable
  
  If its necessary, convert relavant column varible to appropriate data type e.g datetime format,numeric format
  
  Extract the data for year 2009 only and store it as dataframe
  
  Perform all the counting and calculation for above extracted dataframe only as mentioned in below steps
  1. To count total claims, first slice the relavant data and then use groupby function to count value for each patient
  2. To sum total cost of claims, first slice the relavant data and then use groupby function to sum the value for each
     patient
  3. To count unique physicians visited, define the function which will give unique provider count and then use groupby
     function to call the function
  4. To idenitfy most frequent claim diagnosis, first define the function which will give value counts across all the claim
     diagnosis columns
  5. To identify most admitting diagnosis, first slice the relevant data and then use groupby function and value count
     function to identify most frequent admitting diagnosis
  
  After completing above five steps, concotenate the extracted dataframe and export the dataframe into csv format.

## Reading CSV file 

We are going to read csv file using pandas

In [2]:
#Import relevant dependancies
import os
import csv
import pandas as pd
import numpy as np

#File to load
file_to_load = "synpuf_outpatient_claims.csv"

#Read medicare outpatient file and store into pandas dataframe
outpatient_df=pd.read_csv(file_to_load, parse_dates=True,dtype="unicode")

#check first few rows of the data
outpatient_df.head(5)

,Unnamed: 0,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,...,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
0,0,00013D2EFD8E45D1,542192281063886,1,20080904,20080904,2600RA,50.00,0.00,4824842417,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,00016F745862898F,542272281166593,1,20090602,20090602,3901GS,30.00,0.00,2963419753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,00016F745862898F,542282281644416,1,20090623,20090623,3939PG,30.00,0.00,5737807753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0001FDD721E223DC,542642281250669,1,20091011,20091011,3902NU,30.00,0.00,1233847710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,00024B3D2352D2D0,542242281386963,1,20080712,20080712,5200TV,30.00,0.00,9688809345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Check total number of rows, and columns
outpatient_df.shape

(790790, 77)

In [4]:
#check the null values
outpatient_df.isnull().sum()[outpatient_df.isnull().sum()>0]

CLM_FROM_DT      11253
CLM_THRU_DT      11253
AT_PHYSN_NPI     17790
OP_PHYSN_NPI    656357
OT_PHYSN_NPI    533124
                 ...  
HCPCS_CD_41     778122
HCPCS_CD_42     778914
HCPCS_CD_43     779594
HCPCS_CD_44     780234
HCPCS_CD_45     790790
Length: 67, dtype: int64

## Extracting the information for the year 2009 only

We are going to convert relavant columns to appropriate data format and then extracting data for 2009 year.


In [5]:
#Display the columns header
outpatient_df.columns

Index(['Unnamed: 0', 'DESYNPUF_ID', 'CLM_ID', 'SEGMENT', 'CLM_FROM_DT',
       'CLM_THRU_DT', 'PRVDR_NUM', 'CLM_PMT_AMT', 'NCH_PRMRY_PYR_CLM_PD_AMT',
       'AT_PHYSN_NPI', 'OP_PHYSN_NPI', 'OT_PHYSN_NPI',
       'NCH_BENE_BLOOD_DDCTBL_LBLTY_AM', 'ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2',
       'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4', 'ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6',
       'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8', 'ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10',
       'ICD9_PRCDR_CD_1', 'ICD9_PRCDR_CD_2', 'ICD9_PRCDR_CD_3',
       'ICD9_PRCDR_CD_4', 'ICD9_PRCDR_CD_5', 'ICD9_PRCDR_CD_6',
       'NCH_BENE_PTB_DDCTBL_AMT', 'NCH_BENE_PTB_COINSRNC_AMT',
       'ADMTNG_ICD9_DGNS_CD', 'HCPCS_CD_1', 'HCPCS_CD_2', 'HCPCS_CD_3',
       'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6', 'HCPCS_CD_7', 'HCPCS_CD_8',
       'HCPCS_CD_9', 'HCPCS_CD_10', 'HCPCS_CD_11', 'HCPCS_CD_12',
       'HCPCS_CD_13', 'HCPCS_CD_14', 'HCPCS_CD_15', 'HCPCS_CD_16',
       'HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19', 'HCPCS_CD_20',
       'HCPCS_CD_21

In [6]:
#Convert the "CLM_FROM_DT" and "CLM_THRU_DT" into datetime format
outpatient_df["CLM_FROM_DT"] = pd.to_datetime(outpatient_df["CLM_FROM_DT"])
outpatient_df["CLM_THRU_DT"] = pd.to_datetime(outpatient_df["CLM_THRU_DT"])

#Convert dataframe "object" type to numeric type
outpatient_df["CLM_PMT_AMT"]=pd.to_numeric(outpatient_df["CLM_PMT_AMT"])

#Extracting data for only year 2009 from "CLM_FROM_DT"
outpatient_2009=outpatient_df[(outpatient_df["CLM_FROM_DT"]>="2009-01-01")&(outpatient_df["CLM_FROM_DT"]<="2009-12-31")]
outpatient_2009.head(5)

,Unnamed: 0,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,...,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
1,1,00016F745862898F,542272281166593,1,2009-06-02,2009-06-02,3901GS,30.0,0.00,2963419753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,00016F745862898F,542282281644416,1,2009-06-23,2009-06-23,3939PG,30.0,0.00,5737807753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0001FDD721E223DC,542642281250669,1,2009-10-11,2009-10-11,3902NU,30.0,0.00,1233847710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,00024B3D2352D2D0,542442280980633,1,2009-02-12,2009-02-12,5213ZG,80.0,0.00,4199631965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,00024B3D2352D2D0,542422281208985,1,2009-06-27,2009-06-27,5213RM,10.0,0.00,1449018601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
outpatient_2009.shape

(322358, 77)

## 1. Counting Total claims in the year

In [8]:
#Slice the relavant data using column names
df_total_claims=outpatient_2009.loc[:, ["DESYNPUF_ID", "CLM_ID"]]

#Grouping the dataframe by Beneficiary code,and count the "CLM_ID" column values for each patient
total_claims=df_total_claims.groupby("DESYNPUF_ID")["CLM_ID"].count()

#Reseting the index and rename the columns
total_claims=total_claims.reset_index()
total_claims=total_claims.rename(index=str,columns={"DESYNPUF_ID":"PatientID","CLM_ID":"Total claims in the year"})
total_claims.set_index("PatientID",inplace=True)

#Print first few rows to check the extracted values
print(total_claims.head())

                  Total claims in the year
PatientID                                 
00016F745862898F                         2
0001FDD721E223DC                         1
00024B3D2352D2D0                         2
0002F28CE057345B                        12
000308435E3E5B76                         8


## 2. Sum the  Total Costs of Outpatient care

In [9]:
#Slice the relavant data using column names
df_total_cost=outpatient_2009.loc[:, ["DESYNPUF_ID", "CLM_PMT_AMT"]]

#Grouping the dataframe by Beneficiary code,and sum the "CLM_PMT_AMT" values
total_cost=df_total_cost.groupby("DESYNPUF_ID")["CLM_PMT_AMT"].sum()

#Reseting the index and rename the columns
total_cost=total_cost.reset_index()
total_cost=total_cost.rename(index=str,columns={"DESYNPUF_ID":"PatientID","CLM_PMT_AMT":"Total Costs of Outpatient care"})
total_cost.set_index("PatientID",inplace=True)

#Print first few rows to check the extracted values
print(total_cost.head())

                  Total Costs of Outpatient care
PatientID                                       
00016F745862898F                            60.0
0001FDD721E223DC                            30.0
00024B3D2352D2D0                            90.0
0002F28CE057345B                          1810.0
000308435E3E5B76                          1790.0


## 3. Counting the total number of physicians visited including attending, operating, and other

In [10]:
#define the function to get unique value across three columns for each patient
def get_unique(group):
    physician_cols = [group['AT_PHYSN_NPI'], 
                      group['OP_PHYSN_NPI'], 
                      group['OT_PHYSN_NPI']]
    return len(pd.concat(physician_cols).dropna(inplace=False).unique())

#Call the function with grouping by beneficiery code
providers_gp=outpatient_2009.groupby("DESYNPUF_ID").apply(get_unique).reset_index()

#Rename column header
provider_gp=providers_gp.rename(index=str,columns={"DESYNPUF_ID":"PatientID",
                                                   0:"Number of physicians visited(across attending,operating and other)"})
#Reseting the index and print first few rows to check values
provider_gp.set_index("PatientID",inplace=True)
print(provider_gp.head())

                  Number of physicians visited(across attending,operating and other)
PatientID                                                                           
00016F745862898F                                                  2                 
0001FDD721E223DC                                                  1                 
00024B3D2352D2D0                                                  2                 
0002F28CE057345B                                                 17                 
000308435E3E5B76                                                  5                 


## 4. Identify most frequent claim diagnosis for the patient

In [11]:
#Define the function to get most frequent value across multiple columns for each patient
def get_claimdx_frequent(group):
    diagnosis_cols = [group["ICD9_DGNS_CD_1"], group["ICD9_DGNS_CD_2"], 
                      group["ICD9_DGNS_CD_3"], group["ICD9_DGNS_CD_4"],
                      group["ICD9_DGNS_CD_5"], group["ICD9_DGNS_CD_6"],
                      group["ICD9_DGNS_CD_7"], group["ICD9_DGNS_CD_8"],
                      group["ICD9_DGNS_CD_9"], group["ICD9_DGNS_CD_10"]]
    return pd.concat(diagnosis_cols).dropna(inplace=False).value_counts().idxmax()

#Call the function with grouping by beneficiery code
claim_gp=outpatient_2009.groupby("DESYNPUF_ID").apply(get_claimdx_frequent).reset_index()

#Rename column header and reset index
claim_gp=claim_gp.rename(index=str,columns={"DESYNPUF_ID":"PatientID",
                                            0:"Most frequent claim diagnosis for the pateint in that year"})
claim_gp.set_index("PatientID",inplace=True)

#print first few rows to check values
print(claim_gp.head())

                 Most frequent claim diagnosis for the pateint in that year
PatientID                                                                  
00016F745862898F                                              V5832        
0001FDD721E223DC                                              78943        
00024B3D2352D2D0                                               6115        
0002F28CE057345B                                               4280        
000308435E3E5B76                                               4019        


## 5. Identify most frequent admitting diagnosis for the patient

In [12]:
#Slice the relavant data using column names
admitting_df=outpatient_2009.loc[:, ["DESYNPUF_ID","ADMTNG_ICD9_DGNS_CD"]]

#Extract the most frequent value for admitting diagnosis using lamda function
f = lambda x: np.nan if x.isnull().all() else x.value_counts().index[0]
admitting_gp=outpatient_2009.groupby("DESYNPUF_ID")["ADMTNG_ICD9_DGNS_CD"].apply(f).reset_index()

#Rename column header and reset index
admitting_gp=admitting_gp.rename(index=str,columns={"DESYNPUF_ID":"PatientID",
                                                    "ADMTNG_ICD9_DGNS_CD":
                                                    "Most frequent admitting Diagnosis for the patient in that year"})
admitting_gp.set_index("PatientID",inplace=True)

#print first few rows to check values
print(admitting_gp.head())

                 Most frequent admitting Diagnosis for the patient in that year
PatientID                                                                      
00016F745862898F                                                NaN            
0001FDD721E223DC                                              56409            
00024B3D2352D2D0                                              79380            
0002F28CE057345B                                              71947            
000308435E3E5B76                                               2724            


## Concotenate all the dataframe and formatting output csv file

In [13]:
#Concotenate all the dataframe on "PatientID"
data_frame=pd.concat([total_claims,total_cost,provider_gp,claim_gp,admitting_gp],axis=1).reset_index()
data_frame.head()

#convert data_frame to csv file as output
file_name="output_synpuf.csv"
output_csv=data_frame.to_csv(file_name,index=False,encoding='utf-8')